In [6]:
%reload_ext autoreload
%autoreload 2

In [9]:
import pandas as pd
import torch
from nanoGPT.model import GPTConfig
from lightning_training import *

In [10]:
torch.cuda.is_available()

True

In [11]:
elite_database = pd.read_csv("./data/lichess_elite_database_2.csv", nrows=100000, delimiter=";", usecols=["result", "white_elo", "black_elo", "piece_uci"])
# elite_database = elite_database.sample(n = 10**7, random_state=42)
elite_database

,white_elo,black_elo,result,piece_uci
0,2271,2428,1-0,Pd2d4 Ng8f6 Pc2c4 Pe7e6 Nb1c3 Bf8b4 Ng1f3 Ke8g...
1,2271,2428,1-0,Pd2d4 Ng8f6 Pc2c4 Pe7e6 Nb1c3 Bf8b4 Ng1f3 Ke8g...
2,2423,2438,1-0,Pd2d4 Ng8f6 Pc2c4 Pc7c6 Nb1c3 Pd7d5 Pe2e3 Pe7e...
3,2439,2276,1-0,Pe2e4 Pd7d5 Pe4d5 Ng8f6 Pc2c4 Pc7c6 Pd2d4 Pc6d...
4,2347,2430,1-0,Pd2d4 Pd7d5 Pc2c4 Pc7c6 Pc4d5 Pc6d5 Nb1c3 Nb8c...
...,...,...,...,...
99995,2285,2433,1-0,Pd2d4 Pg7g6 Pe2e4 Bf8g7 Nb1c3 Pd7d6 Bc1e3 Pa7a...
99996,2503,2508,1-0,Pe2e4 Pc7c5 Nb1c3 Pd7d6 Pf2f4 Pg7g6 Ng1f3 Bf8g...
99997,2310,2452,0-1,Pe2e4 Pc7c6 Pd2d4 Pd7d5 Pe4d5 Pc6d5 Bf1d3 Nb8c...
99998,2450,2312,1-0,Pd2d4 Pf7f5 Pg2g3 Ng8f6 Bf1g2 Pd7d6 Ng1f3 Pg7g...


In [12]:
# data_module = WeightedGamesDataModule(elite_database, test_size=0.01, num_workers=16)
games = list(elite_database.piece_uci)
data_module = GamesDataModule(games, batch_size=64, test_size=0.01, num_workers=12)

In [13]:
model_config = GPTConfig(
    block_size=301,
    vocab_size=len(data_module.tokenizer.vocab),
    n_layer=4,
    n_head=4,
    n_embd=256,
    bias=False,
)

# pl_model = LightningGPTWeighted(model_config)
pl_model = LightningGPT(model_config)

number of parameters: 4.19M


In [14]:
torch.cuda.is_available()

True

In [17]:
tensorboard_logger = pl.loggers.TensorBoardLogger(save_dir="./lightning_logs/strong_play/", name=f"W1-D0.5-L0")

trainer = pl.Trainer(
    accelerator="gpu",
    max_epochs=4,
    # callbacks=[pl.callbacks.RichProgressBar()],
    logger=tensorboard_logger,
    # default_root_dir=
    # fast_dev_run=True
)

ModuleNotFoundError: `RichProgressBar` requires `rich` >= 10.2.2. Install it by running `pip install -U rich`.

In [16]:
trainer.fit(
    model=pl_model,
    datamodule=data_module,
)

NameError: name 'trainer' is not defined

In [15]:
torch.cuda.is_available()

False

In [19]:
data_loader = data_module.train_dataloader()
model = pl_model.model
model.cuda()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [21]:
for i, (x, y) in enumerate(data_loader):
    print(i)
    optimizer.zero_grad()
    x = x.cuda()
    y = y.cuda()
    out, loss = model(x, y)
    loss.backward()
    optimizer.step()



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67


KeyboardInterrupt: 